# Analisis

In [8]:
import numpy as np
from sklearn import linear_model
from matplotlib import pyplot as plt

In [38]:
%run codigos/mv_to_ep.ipynb
%run codigos/R_Fresnel.ipynb

In [9]:
#-----------------------------------
## Parámetros globales de la antena
#-----------------------------------

Fs = 23.328e9 #Frecuencia de muestreo [Hz] 
Ts = 1/Fs
BW = 1.5e9 
corte = 150
Nfft= 1024 # nro de frecs -> conviene que sea multiplo de 2 para calcular la FFT de manera eficiente. 
#Tiene que ser mayor a la longitud de la señal (285 para 2m, 439 para 3m)

#------------------------------
#Elegir frecuencia portadora
#------------------------------
portadora = input('portadora? (Fc7 o Fc8)')

if portadora == 'Fc7':
    fc = 7.290e9 
    Fc = '7.29GHz'
elif portadora == 'Fc8':
    fc = 8.748e9
    Fc = '8.74GHz'
else:
    print('Error: portadora debe ser Fc7 o Fc8')

## Importo variables

In [10]:
import pickle
if portadora == 'Fc8':
    with open('mayo_Fc8.pkl','rb') as g1:
        Mayo8 = pickle.load(g1)
    with open('junio_Fc8.pkl','rb') as g2:
        Junio8 = pickle.load(g2)
    with open('julio_Fc8.pkl','rb') as g3:
        Julio8 = pickle.load(g3)
    with open('agosto_Fc8.pkl','rb') as g4:
        Agosto8 = pickle.load(g4)

    print(Mayo8.keys())
    print(Junio8.keys())
    print(Julio8.keys())
    print(Agosto8.keys())
else:
    with open('mayo_Fc7.pkl','rb') as g1:
        Mayo7 = pickle.load(g1)
    with open('junio_Fc7.pkl','rb') as g2:
        Junio7 = pickle.load(g2)
    with open('julio_Fc7.pkl','rb') as g3:
        Julio7 = pickle.load(g3)
    with open('agosto_Fc7.pkl','rb') as g4:
        Agosto7 = pickle.load(g4)
    print(Mayo7.keys())
    print(Junio7.keys())
    print(Julio7.keys())
    print(Agosto7.keys())

dict_keys(['h1_may', 'h2_may', 'f_may', 'S1_may', 'S2_may'])
dict_keys(['h1_jun', 'h2_jun', 'f_jun', 'S1_jun', 'S2_jun', 'hz_jun', 'Z2_jun', 'tita_jun'])
dict_keys(['h1_jul', 'h2_jul', 'f_jul', 'S1_jul', 'S2_jul', 'hz_jul', 'Z2_jul', 'tita_jul'])
dict_keys(['h_ag', 'f_ag', 'S1A_ag', 'S1B_ag', 'S2_ag', 'M2_ag', 'hz_ag', 'Z2_ag', 'MZ2_ag', 'tita_ag'])


In [50]:
print(Mayo8.get('S1_may')[2,:])

[-1.34033528e-02-5.02637104e-03j -9.59640280e-03-3.70414463e-03j
 -5.64206435e-03-4.89325762e-03j -2.86208938e-03-8.18393809e-03j
 -2.34382533e-03-1.26600102e-02j -4.37641993e-03-1.65020592e-02j
 -7.47733655e-03-1.84195298e-02j -1.03131169e-02-1.91415045e-02j
 -1.33628073e-02-1.92405551e-02j -1.64447938e-02-1.76749472e-02j
 -1.76486426e-02-1.47485808e-02j -1.68855919e-02-1.28147656e-02j
 -1.62906287e-02-1.22351852e-02j -1.59100980e-02-1.13466950e-02j
 -1.42380358e-02-1.08808088e-02j -1.27157836e-02-1.26673249e-02j
 -1.36311578e-02-1.50259381e-02j -1.54710026e-02-1.55755317e-02j
 -1.63415146e-02-1.56824580e-02j -1.78996881e-02-1.63890651e-02j
 -2.04512445e-02-1.52161033e-02j -2.04870267e-02-1.20535942e-02j
 -1.74906606e-02-1.10403483e-02j -1.53022084e-02-1.32221922e-02j
 -1.47837874e-02-1.59484370e-02j -1.45588163e-02-1.98062006e-02j
 -1.75319092e-02-2.58623795e-02j -2.56880242e-02-2.92519178e-02j
 -3.34764586e-02-2.68426255e-02j -3.77409247e-02-2.40926184e-02j
 -4.46177526e-02-2.295351

In [11]:
#chequeo que las frecuencias sean iguales
# fm = Mayo8.get('f_may')
# fjn = Junio8.get('f_jun')
# fjl = Julio8.get('f_jul')
# fa = Agosto8.get('f_ag')

# L = [fm,fjn,fjl,fa]
# out = (np.diff(np.vstack(L).reshape(len(L),-1),axis=0)==0).all()
# print('todas las frecuencias son siempre iguales?',out)

# if out:
#     f = fm
#     print('renombro las frecuencias como f')
# else:
#     print('diferentes')

f = Mayo8.get('f_may')

In [14]:
with open('codigos/hydra_may.pkl','rb') as g5:
        hydra_may = pickle.load(g5)
with open('codigos/hydra_jun.pkl','rb') as g6:
        hydra_jun = pickle.load(g6)
with open('codigos/hydra_jul.pkl','rb') as g7:
        hydra_jul = pickle.load(g7)
with open('codigos/hydra_ag.pkl','rb') as g8:
        hydra_ag = pickle.load(g8)

## 


# Valores para el modelo de suelo

In [39]:
mv_min_med = np.min([np.min(hydra_may.get('mv_med_may')), np.min(hydra_jun.get('mv_med_jun')), np.min(hydra_jul.get('mv_med_jul')), np.min(hydra_ag.get('mv_med_ag'))])
mv_max_med = np.max([np.max(hydra_may.get('mv_med_may')), np.max(hydra_jun.get('mv_med_jun')), np.max(hydra_jul.get('mv_med_jul')), np.max(hydra_ag.get('mv_med_ag'))])
print(mv_min_med,mv_max_med)

0.0 0.376


In [40]:
#grilla de valores para MV
mv_min, mv_max, n_mv = 0, 0.55, 100
MV = np.linspace(mv_min,mv_max,n_mv)

Sand = 0.5
Clay = 0.5

ep_teo = mv_to_ep(MV,Sand,Clay,0)['ep']

#usando el error
#ep_std = 0.01 #std para estimar el error en ep del modelo teórico
#ep_teo = mv_to_ep(MV,Sand,Clay,ep_std)['ep']
#ep_error = mod.mv_to_ep(MV,Sand, Clay,ep_std)['ep_error']

ep_min, ep_max = np.min(ep_teo), np.max(ep_teo)
print('ep min:', ep_min)
print('ep max:', ep_max)


ep min: (2.007-0.198j)
ep max: (28.246537500000002+6.16719125j)


## medio semi-infinito

In [56]:
# vector para guardar las métricas de error
h1_may = Mayo8.get('h1_may')
S1_may = Mayo8.get('S1_may')
e_mae = np.zeros((len(h1_may),n_mv),dtype=float)
S1teo_may = np.zeros((len(h1_may),n_mv), dtype=complex)

print(np.shape(S1_may)[1])

#obs: len(ep_teo)=len(MV)=n_mv
for hm in range(len(h1_may)):
    for i in range(n_mv):
        ep = ep_teo[i]
    
        #coef de reflexion depende solo de ep
        R = R_1capa(ep)
        S1teo_may[hm,i] = S1_may[hm,:]*R

        ##error
        e_mae[hm,i] = np.sum(np.abs(np.abs(S1teo_may[hm])-np.abs(S1_may[hm])))/np.shape(S1_may)[1]

    ind = np.zeros(len(h1_may))
    ind[hm] = np.argmin(e_mae[hm])

    # mv_op = MV[ind]
    # ep_op = ep_teo[ind]
    # S_op = Scal_avg_fpos*R_1capa(ep_op)
    # error_mae = e_mae[ind]

153


ValueError: could not broadcast input array from shape (153,) into shape (100,)